<a href="https://colab.research.google.com/github/alexander-toschev/ai-tools/blob/main/tasks/Task1_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# DO NOT MODIFY  !!!
# DO NOT EXECUTE !!!
!pip install --upgrade gspread pandas google-auth
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
from IPython.display import display
import random
# Authenticate and create the PyDrive client.
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)



In [25]:
# FILL THIS
student_name = "ELON MUSK"
group_id = "11-101"


In [26]:
# DO NOT MODIFY  !!!
# DO NOT EXECUTE !!!
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1Kfxj2eDFl7xQnXw7Fpb9bwghc65o8xf--VpNxrdWHaY/edit?gid=0#gid=0"
sh = gc.open_by_url(SPREADSHEET_URL)
worksheet = sh.sheet1

# Ensure header row exists
if not worksheet.get_all_values():
    worksheet.append_row(["Student Name", "Group","TaskID", "Score"])


In [27]:
# MAIN NOTEBOOK GOES HERE
task_id = "Task1_PyTorch"
score = 0
max_score = 15

In [28]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [29]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


In [30]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# Modelling

In [31]:
# mps for Mac
global device
device = "cuda"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To train a model, we need a loss function and an optimizer.

In [32]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model’s parameters.

In [33]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [34]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return  100*correct, test_loss

In [35]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Epoch 1
-------------------------------
loss: 2.302124  [   64/60000]
loss: 2.290845  [ 6464/60000]
loss: 2.273964  [12864/60000]
loss: 2.266565  [19264/60000]
loss: 2.242912  [25664/60000]
loss: 2.216395  [32064/60000]
loss: 2.221253  [38464/60000]
loss: 2.186315  [44864/60000]
loss: 2.182127  [51264/60000]
loss: 2.141618  [57664/60000]
Test Error: 
 Accuracy: 43.2%, Avg loss: 2.142539 

Epoch 2
-------------------------------
loss: 2.151726  [   64/60000]
loss: 2.143460  [ 6464/60000]
loss: 2.087738  [12864/60000]
loss: 2.107926  [19264/60000]
loss: 2.041686  [25664/60000]
loss: 1.991503  [32064/60000]
loss: 2.012198  [38464/60000]
loss: 1.931440  [44864/60000]
loss: 1.935564  [51264/60000]
loss: 1.854998  [57664/60000]
Test Error: 
 Accuracy: 55.4%, Avg loss: 1.862655 

Epoch 3
-------------------------------
loss: 1.894671  [   64/60000]
loss: 1.864006  [ 6464/60000]
loss: 1.753084  [12864/60000]
loss: 1.803132  [19264/60000]
loss: 1.677571  [25664/60000]
loss: 1.645940  [32064/600

# Task 1
Build graphic for the learning process.

1. Load/Save models

In [36]:
def createModel():
  # YOUR CODE START 2 lines
  # create model
  model =

  return model
  # YOUR CODE END

In [37]:
def checkClasses(model):
  classes = [
      "T-shirt/top",
      "Trouser",
      "Pullover",
      "Dress",
      "Coat",
      "Sandal",
      "Shirt",
      "Sneaker",
      "Bag",
      "Ankle boot",
  ]

  model.eval()
  # YOUR CODE START 4 lines
  # load 1st class
  x, y =
  with torch.no_grad():
      # predict
      pred =
      # return predicted, actual
      predicted, actual =
  return predicted, actual

In [38]:
import unittest
import cv2
import numpy as np

class TestImageFilters(unittest.TestCase):

    def test_model_creation(self):
        # Загрузка изображения
        model = createModel()
        predicted, actual = checkClasses(model)
        self.assertEqual(predicted, actual, "Prediction wrong")
        global score
        score=score+5

# Run unit tests
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
    print(f"Student Score: {score}/{max_score}")

.
----------------------------------------------------------------------
Ran 1 test in 0.010s

OK


Student Score: 5/15


In [39]:
model = createModel()
predicted, actual = checkClasses(model)
print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


# Task 2. Improve training

In [40]:
# `YOUR CODE HERE, Improve model to get over 80% accuracy
def improveModel():
  epochs = 10
  global device

  for t in range(epochs):
      print(f"Epoch {t+1}\n-------------------------------")
      train(train_dataloader, model, loss_fn, optimizer)
      size = len(train_dataloader.dataset)
      model.train()
      for batch, (X, y) in enumerate(train_dataloader):
          X, y = X.to(device), y.to(device)

          # Compute prediction error
          pred = model(X)
          loss = loss_fn(pred, y)

          # Backpropagation
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          if batch % 100 == 0:
              loss, current = loss.item(), (batch + 1) * len(X)
              print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
      acc, loss = test(test_dataloader, model, loss_fn)
  return acc, loss

In [41]:
import unittest
import cv2
import numpy as np

class TestImageFilters(unittest.TestCase):

    def test_model_improve(self):
        acc, loss = improveModel()
        self.assertGreaterEqual(acc, 80, "Model is not accurate")
        global score
        score=score+10

# Run unit tests
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
    print(f"Student Score: {score}/{max_score}")

E
ERROR: test_model_improve (__main__.TestImageFilters.test_model_improve)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-41-90197d78d99a>", line 8, in test_model_improve
    acc, loss = improveModel()
                ^^^^^^^^^^^^^^
  File "<ipython-input-40-bcca0ae635a9>", line 8, in improveModel
    train(train_dataloader, model, loss_fn, optimizer)
  File "<ipython-input-33-f5c3e8696046>", line 8, in train
    pred = model(X)
           ^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1750, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-31-2ae6659f9425>", line 21, in forward
    logits = self.lin

Epoch 1
-------------------------------
Student Score: 5/15


In [42]:
# DO NOT MODIFY  !!!
# DO NOT EXECUTE !!!
# Save the result to Google Sheets
worksheet.append_row([student_name,group_id, task_id, score])

print(f"Test completed! {student_name}, your score is {score}/{max_score}.")

Test completed! ELON MUSK, your score is 5/15.
